# Capítulo 7: Creando aplicaciones de chat
## Guía rápida de la API de OpenAI

Este cuaderno está adaptado del [Repositorio de ejemplos de Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), que incluye cuadernos que acceden a los servicios de [Azure OpenAI](notebook-azure-openai.ipynb).

La API de OpenAI para Python también funciona con los modelos de Azure OpenAI, con algunas modificaciones. Puedes obtener más información sobre las diferencias aquí: [Cómo cambiar entre los endpoints de OpenAI y Azure OpenAI con Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Descripción general  
"Los modelos de lenguaje grandes son funciones que asignan texto a texto. Dado un texto de entrada, un modelo de lenguaje grande intenta predecir el texto que seguirá"(1). Este cuaderno de inicio rápido presentará a los usuarios conceptos generales sobre los LLM, los requisitos principales de los paquetes para comenzar con AML, una introducción sencilla al diseño de prompts y varios ejemplos breves de diferentes casos de uso.


## Tabla de Contenidos  

[Descripción general](../../../../07-building-chat-applications/python)  
[Cómo usar el servicio de OpenAI](../../../../07-building-chat-applications/python)  
[1. Crear tu servicio de OpenAI](../../../../07-building-chat-applications/python)  
[2. Instalación](../../../../07-building-chat-applications/python)    
[3. Credenciales](../../../../07-building-chat-applications/python)  

[Casos de uso](../../../../07-building-chat-applications/python)    
[1. Resumir texto](../../../../07-building-chat-applications/python)  
[2. Clasificar texto](../../../../07-building-chat-applications/python)  
[3. Generar nuevos nombres de productos](../../../../07-building-chat-applications/python)  
[4. Ajustar un clasificador](../../../../07-building-chat-applications/python)  

[Referencias](../../../../07-building-chat-applications/python)


### Crea tu primer prompt  
Este breve ejercicio te dará una introducción básica para enviar prompts a un modelo de OpenAI para una tarea sencilla: "resumir".


**Pasos**:  
1. Instala la biblioteca de OpenAI en tu entorno de Python  
2. Carga las bibliotecas auxiliares estándar y configura tus credenciales de seguridad habituales para el servicio de OpenAI que hayas creado  
3. Elige un modelo para tu tarea  
4. Crea un prompt sencillo para el modelo  
5. ¡Envía tu solicitud a la API del modelo!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Encontrar el modelo adecuado  
Los modelos GPT-3.5-turbo o GPT-4 pueden comprender y generar lenguaje natural.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Diseño de Prompts  

"La magia de los grandes modelos de lenguaje es que, al ser entrenados para minimizar este error de predicción sobre enormes cantidades de texto, los modelos terminan aprendiendo conceptos útiles para estas predicciones. Por ejemplo, aprenden conceptos como"(1):

* cómo se escribe correctamente
* cómo funciona la gramática
* cómo parafrasear
* cómo responder preguntas
* cómo mantener una conversación
* cómo escribir en muchos idiomas
* cómo programar
* etc.

#### Cómo controlar un gran modelo de lenguaje  
"De todos los insumos de un gran modelo de lenguaje, por mucho el más influyente es el prompt de texto(1).

Se puede guiar a los grandes modelos de lenguaje para que generen resultados de varias maneras:

Instrucción: Dile al modelo lo que quieres
Completado: Haz que el modelo complete el inicio de lo que deseas
Demostración: Muéstrale al modelo lo que quieres, ya sea con:
Algunos ejemplos en el prompt
Cientos o miles de ejemplos en un conjunto de datos de entrenamiento para ajuste fino"



#### Hay tres pautas básicas para crear prompts:

**Muestra y explica**. Deja claro lo que quieres, ya sea a través de instrucciones, ejemplos o una combinación de ambos. Si quieres que el modelo ordene una lista de elementos alfabéticamente o que clasifique un párrafo según su sentimiento, muéstrale que eso es lo que buscas.

**Proporciona datos de calidad**. Si intentas crear un clasificador o que el modelo siga un patrón, asegúrate de que haya suficientes ejemplos. Revisa bien tus ejemplos: el modelo suele ser lo suficientemente inteligente como para detectar errores ortográficos básicos y darte una respuesta, pero también podría asumir que es intencional y eso puede afectar la respuesta.

**Revisa tu configuración.** Los parámetros temperature y top_p controlan cuán determinista es el modelo al generar una respuesta. Si le pides una respuesta donde solo hay una opción correcta, deberías poner estos valores bajos. Si buscas respuestas más variadas, podrías configurarlos más altos. El error más común con estos parámetros es asumir que controlan la "inteligencia" o "creatividad" del modelo.


Fuente: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Resumir texto  
#### Desafío  
Resume un texto añadiendo un 'tl;dr:' al final de un pasaje. Observa cómo el modelo entiende cómo realizar varias tareas sin instrucciones adicionales. Puedes probar con indicaciones más descriptivas que tl;dr para modificar el comportamiento del modelo y personalizar el resumen que recibes(3).  

Trabajos recientes han demostrado mejoras significativas en muchas tareas y pruebas de PLN al preentrenar con un gran corpus de texto y luego ajustar el modelo para una tarea específica. Aunque normalmente la arquitectura es independiente de la tarea, este método aún requiere conjuntos de datos de ajuste fino con miles o decenas de miles de ejemplos. En cambio, los humanos suelen poder realizar una nueva tarea de lenguaje con solo unos pocos ejemplos o instrucciones simples, algo con lo que los sistemas actuales de PLN todavía tienen dificultades. Aquí mostramos que aumentar la escala de los modelos de lenguaje mejora notablemente el rendimiento independiente de la tarea y con pocos ejemplos, llegando en ocasiones a ser tan competitivos como los enfoques previos de ajuste fino más avanzados.  

Tl;dr


# Ejercicios para varios casos de uso  
1. Resumir texto  
2. Clasificar texto  
3. Generar nuevos nombres de productos


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Clasificar texto  
#### Desafío  
Clasifica elementos en categorías que se proporcionan en el momento de la inferencia. En el siguiente ejemplo, proporcionamos tanto las categorías como el texto a clasificar en el prompt (*playground_reference).

Consulta del cliente: Hola, una de las teclas de mi teclado de portátil se rompió recientemente y necesito un reemplazo:

Categoría clasificada:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Genera nuevos nombres de productos
#### Desafío
Crea nombres de productos a partir de palabras de ejemplo. Aquí incluimos en la indicación información sobre el producto para el que vamos a generar nombres. También proporcionamos un ejemplo similar para mostrar el patrón que queremos recibir. Además, hemos configurado el valor de temperatura alto para aumentar la aleatoriedad y obtener respuestas más innovadoras.

Descripción del producto: Una máquina para hacer batidos en casa
Palabras clave: rápido, saludable, compacto.
Nombres de productos: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Descripción del producto: Un par de zapatos que se adapta a cualquier talla de pie.
Palabras clave: adaptable, ajuste, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referencias  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Ejemplos de OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Mejores prácticas para ajustar GPT-3 y clasificar texto](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Para más ayuda  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Colaboradores
* Louis Li



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de ningún malentendido o interpretación errónea que surja del uso de esta traducción.
